In [12]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver

ret = load_dotenv()

# returns True if .env loaded successfully
print(ret)

True


In [13]:
model = ChatOpenAI(model_name="gpt-4o-mini")
model_response = model.invoke("Tell me about Dexcom Stelo")
print(model_response)

content="As of my last knowledge update in October 2023, Dexcom Stelo is a continuous glucose monitoring (CGM) system developed by Dexcom, designed to help individuals with diabetes manage their blood glucose levels more effectively. The system is a part of Dexcom's innovative approach to diabetes management, providing real-time glucose data to users through a small sensor placed under the skin.\n\nSome key features of the Dexcom Stelo system include:\n\n1. **Real-time Glucose Monitoring**: The Stelo system continuously tracks glucose levels and transmits data to a compatible device, allowing users to monitor their blood sugar levels in real time.\n\n2. **Alerts and Notifications**: Users can set customizable alerts and notifications for high or low glucose levels, which can help in timely intervention to prevent hyperglycemia or hypoglycemia.\n\n3. **Data Sharing**: The system often features the ability to share glucose data with caregivers, family members, or healthcare providers via

In [16]:
# define the function that calls the model
def call_model(state: MessagesState):
    # state["messages"] is a list of all historical messages
    updated_messages = model.invoke(state["messages"])

    # models response appended to the existing list of messages
    return {"messages": updated_messages}

In [18]:
# workflow is a stategraph, keeps track of the state of the application
# we need to use a schema, MessagesState helps to keep track a list
# of messages
workflow = StateGraph(MessagesState)
workflow.add_node("model_node", call_model)

# after START, run the model node which calls the model
workflow.add_edge(START, "model_node")

In [19]:
# real apps will use Postgres or SQlite checkpointers
# for experimentation use in memory checkpointer
# checkpointers store snapshots of past conversation
memory = MemorySaver()
app = workflow.compile(memory)

In [21]:
# app is now a runnable bject so has invoke method
# because schema used is MessagesState, input expected
# is dict of {'messages': ""}

chat1 = {'configurable': {'thread_id': 1}}
app.invoke({"messages": "Hi My name is Mahmud!"}, config=chat1)

{'messages': [HumanMessage(content='Hi My name is Mahmud!', additional_kwargs={}, response_metadata={}, id='5c6081e0-dedc-47c5-8236-bf175c5c3133'),
  AIMessage(content='Hi Mahmud! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-954d42d8-6d7b-43ae-a408-a2e6028a8318-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [22]:
# look at the reply. the whole conversation is here
app.invoke({"messages": "What is my name!"}, config=chat1)

{'messages': [HumanMessage(content='Hi My name is Mahmud!', additional_kwargs={}, response_metadata={}, id='5c6081e0-dedc-47c5-8236-bf175c5c3133'),
  AIMessage(content='Hi Mahmud! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-954d42d8-6d7b-43ae-a408-a2e6028a8318-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='What is my name!', additional_kwargs={}, response_metadata={}, 

In [24]:
output = app.invoke(None, config=chat1)
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi My name is Mahmud!
================================== Ai Message ==================================

Hi Mahmud! How can I assist you today?
================================ Human Message =================================

What is my name!
================================== Ai Message ==================================

Your name is Mahmud! How can I help you today, Mahmud?


In [25]:
# can the app support multiple independant conversations?
chat2 = {'configurable': {'thread_id': 2}}
app.invoke({"messages": "What is my name!"}, config=chat2)["messages"][-1].content

"I don't know your name unless you tell me! How can I assist you today?"

This time the app doesnt know my name confirming this is an entirely different chat

In [26]:
# make the app more interactive
def chatbot(chat_id: int):
    config = {"configurable": {"thread_id": chat_id}}

    while True:
        user_input = input("User: ")
        if user_input in ('exit', 'quit'):
            print("Goodbye")
            break
        else:
            output = app.invoke({"messages": user_input}, config=config)
            ai_response = output["messages"][-1].content
            print("AI response:", ai_response)


In [27]:
chatbot(4)

AI response: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
Goodbye
